# ELO Rating Comparison Across Models

Compare Bradley-Terry ELO ratings derived from each model's predicted win probabilities.
Each model's predictions are processed through the turn_predicted pipeline:
weighted strength → relative strength → winner adjustment → civ-adjusted OLS → BT ratings.

In [1]:
%matplotlib inline
import sys
import os
sys.path.insert(0, os.path.abspath(".."))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from itertools import combinations
from scipy.stats import spearmanr

from plot_utilities import (
    setup_notebook_display,
    get_player_color,
    plot_matchup_heatmap,
)
from ratings import calculate_ratings_bt
from loader import load_models, MODEL_ORDER, MODEL_COLORS, OUTPUT_DIR

setup_notebook_display(figsize=(14, 7))

# Load all models except naive, in canonical order
all_models = load_models()
models = {k: v for k, v in all_models.items() if k != "naive"}
model_names = [m for m in MODEL_ORDER if m in models]
pred_files = {name: OUTPUT_DIR / f"{name}_predictions.csv" for name in model_names}
print(f'\nUsing {len(model_names)} models: {model_names}')

Loaded naive: 1,875,256 rows
Loaded score: 1,875,256 rows
Loaded baseline: 1,875,256 rows
Loaded xgboost: 1,875,256 rows
Loaded mlp: 1,875,256 rows
Loaded grouped_mlp: 1,875,256 rows
Loaded interaction_mlp: 1,875,256 rows
Loaded attention_mlp: 1,875,256 rows

Using 7 models: ['score', 'baseline', 'xgboost', 'mlp', 'grouped_mlp', 'interaction_mlp', 'attention_mlp']


In [2]:
# Import the canonical strength pipeline instead of duplicating it
from ratings.iterative_bt import prepare_strength_data

def compute_strength_df(csv_path):
    """Thin wrapper: run prepare_strength_data and return only strength_df."""
    strength_df, _, _ = prepare_strength_data(str(csv_path), verbose=False)
    return strength_df

In [3]:
# Run the turn_predicted pipeline on all models
strength_dfs = {}
for name in model_names:
    print(f'Processing {name}...')
    strength_dfs[name] = compute_strength_df(pred_files[name])
    n_games = strength_dfs[name]['game_id'].nunique()
    n_players = strength_dfs[name]['player_type'].nunique()
    print(f'  → {n_games} games, {n_players} player types\n')

print(f'Done: {len(strength_dfs)} models processed.')

Processing score...
  → 357 games, 16 player types

Processing baseline...
  → 357 games, 16 player types

Processing xgboost...
  → 357 games, 16 player types

Processing mlp...
  → 357 games, 16 player types

Processing grouped_mlp...
  → 357 games, 16 player types

Processing interaction_mlp...
  → 357 games, 16 player types

Processing attention_mlp...
  → 357 games, 16 player types

Done: 7 models processed.


In [4]:
# Compute Bradley-Terry ELO ratings for all models
# This calls R 7 times — expect ~1-3 minutes total
rating_dfs = {}
for name, sdf in strength_dfs.items():
    print(f'Computing BT ratings for {name}...')
    rating_dfs[name] = calculate_ratings_bt(sdf, verbose=False)
    top = rating_dfs[name].iloc[0]
    bot = rating_dfs[name].iloc[-1]
    print(f'  → ELO range: {bot["elo"]:.0f} – {top["elo"]:.0f} '
          f'(spread: {top["elo"] - bot["elo"]:.0f})\n')

print('All ratings computed.')

Computing BT ratings for score...
  → ELO range: 1385 – 1500 (spread: 115)

Computing BT ratings for baseline...
  → ELO range: 1370 – 1514 (spread: 144)

Computing BT ratings for xgboost...
  → ELO range: 1351 – 1505 (spread: 154)

Computing BT ratings for mlp...
  → ELO range: 1350 – 1502 (spread: 152)

Computing BT ratings for grouped_mlp...
  → ELO range: 1332 – 1502 (spread: 169)

Computing BT ratings for interaction_mlp...
  → ELO range: 1343 – 1503 (spread: 159)

Computing BT ratings for attention_mlp...
  → ELO range: 1342 – 1502 (spread: 161)

All ratings computed.


In [5]:
# Build combined ELO comparison DataFrame
elo_records = []
for name, rdf in rating_dfs.items():
    for _, row in rdf.iterrows():
        elo_records.append({
            'player_type': row['player_type'],
            'model': name,
            'elo': row['elo'],
        })

elo_long = pd.DataFrame(elo_records)
elo_wide = elo_long.pivot(index='player_type', columns='model', values='elo')
elo_wide = elo_wide[model_names]  # consistent column order (MODEL_ORDER sans naive)

# Summary statistics across models
elo_wide['mean_elo'] = elo_wide[model_names].mean(axis=1)
elo_wide['elo_std'] = elo_wide[model_names].std(axis=1)
elo_wide['elo_range'] = elo_wide[model_names].max(axis=1) - elo_wide[model_names].min(axis=1)
elo_wide = elo_wide.sort_values('mean_elo', ascending=False)

display(elo_wide.style.format('{:.0f}').background_gradient(
    subset=model_names, cmap='RdYlGn', axis=None, vmin=1300, vmax=1600
))

## Consensus ELO

In [6]:
# Viz 6: Consensus forest plot
consensus_order = elo_wide.sort_values('mean_elo', ascending=True).index

fig, ax = plt.subplots(figsize=(12, 10))
y_pos = np.arange(len(consensus_order))

for i, pt in enumerate(consensus_order):
    mean_val = elo_wide.loc[pt, 'mean_elo']
    min_val = elo_wide.loc[pt, model_names].min()
    max_val = elo_wide.loc[pt, model_names].max()
    color = get_player_color(pt)

    # Horizontal range bar
    ax.plot([min_val, max_val], [i, i], color=color, linewidth=3, alpha=0.4)
    # Mean point
    ax.scatter(mean_val, i, color=color, s=100, zorder=5, edgecolors='black', linewidth=0.5)

ax.axvline(1500, color='gray', linestyle='--', linewidth=1.5, label='Vanilla baseline (1500)')
ax.set_yticks(y_pos)
ax.set_yticklabels(consensus_order)
ax.set_xlabel('ELO Rating', fontsize=12)
ax.set_title('Consensus ELO with Model Agreement Range', fontsize=14, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

C:\Users\John Chen\AppData\Local\Temp\ipykernel_176572\2498885545.py:27: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


## Key Findings

In [7]:
# Summary statistics
print('=' * 60)
print('CONSENSUS RANKING (sorted by mean ELO across all models)')
print('=' * 60)
for rank, (pt, row) in enumerate(elo_wide.iterrows(), 1):
    print(f'{rank:>3}. {pt:<25} ELO: {row["mean_elo"]:>7.0f} ± {row["elo_std"]:>5.0f}  '
          f'(range: {row["elo_range"]:>4.0f})')

# Consistently above/below Vanilla
print('\n' + '=' * 60)
print('CONSISTENT ACROSS ALL MODELS')
print('=' * 60)
always_above = [pt for pt in elo_wide.index
                if (elo_wide.loc[pt, model_names] > 1500).all() and pt != 'Vanilla']
always_below = [pt for pt in elo_wide.index
                if (elo_wide.loc[pt, model_names] < 1500).all() and pt != 'Vanilla']

if always_above:
    print(f'\nAlways ABOVE Vanilla (1500): {", ".join(always_above)}')
else:
    print('\nNo player type is above Vanilla in ALL models.')

if always_below:
    print(f'Always BELOW Vanilla (1500): {", ".join(always_below)}')
else:
    print('No player type is below Vanilla in ALL models.')

# Most disagreed
print('\n' + '=' * 60)
print('MOST MODEL DISAGREEMENT (largest ELO range)')
print('=' * 60)
top_disagree = elo_wide.nlargest(5, 'elo_range')
for pt, row in top_disagree.iterrows():
    print(f'  {pt:<25} range: {row["elo_range"]:>4.0f} ELO points')

# Overall model agreement
print('\n' + '=' * 60)
print('OVERALL MODEL AGREEMENT')
print('=' * 60)
rank_data = elo_wide[model_names].rank(ascending=False).astype(int)
corr_values = []
for i, j in combinations(range(len(model_names)), 2):
    r1 = rank_data[model_names[i]].values
    r2 = rank_data[model_names[j]].values
    corr, _ = spearmanr(r1, r2)
    corr_values.append(corr)

print(f'  Mean pairwise Spearman ρ: {np.mean(corr_values):.3f}')
print(f'  Min pairwise Spearman ρ:  {np.min(corr_values):.3f}')
print(f'  Max pairwise Spearman ρ:  {np.max(corr_values):.3f}')

CONSENSUS RANKING (sorted by mean ELO across all models)
  1. Sonnet-4.5-Simple         ELO:    1502 ±     6  (range:   19)
  2. Vanilla                   ELO:    1500 ±     0  (range:    0)
  3. Kimi-K2.5-Briefed         ELO:    1496 ±     7  (range:   22)
  4. Qwen-3.5-Briefed          ELO:    1489 ±     7  (range:   19)
  5. Minimax-M2.5-Briefed      ELO:    1486 ±     4  (range:   10)
  6. Minimax-M2.5-Simple       ELO:    1471 ±     5  (range:   16)
  7. Kimi-K2.5-Simple          ELO:    1445 ±     8  (range:   23)
  8. Deepseek-3.2-Briefed      ELO:    1427 ±     4  (range:   11)
  9. GLM-4.7-Simple            ELO:    1422 ±     6  (range:   14)
 10. GPT-OSS-120B-Simple       ELO:    1409 ±     8  (range:   22)
 11. Qwen-3.5-Simple           ELO:    1408 ±    12  (range:   36)
 12. Sonnet-4.5-Briefed        ELO:    1401 ±     6  (range:   19)
 13. GPT-OSS-120B-Briefed      ELO:    1401 ±     7  (range:   19)
 14. Deepseek-3.2-Simple       ELO:    1399 ±    11  (range:   29)
 15. 